In [13]:
import psycopg2
import logging

DB_NAME = 'blockchain'
DB_USER = 'postgres'
DB_PASSWORD = 'auth2020'
DB_HOST = 'localhost'
DB_PORT = '5432'


try:
    # Create a PostgreSQL connection
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()
    print("Connection established.")  # Log successful connection (you can modify this)

    # Configure logging to write to a log file
    logging.basicConfig(filename='log.txt', level=logging.INFO)
    logging.info("Connection established with Database.")  # Log the same message to the file


    # Additional code for database operations goes here...

except Exception as err:
    # Handle exceptions
    print("Unable to connect to database.")  # Log error (you can modify this)
    print(err)  # Print the specific error message

    # Optionally, write the error message to a file (error.txt)
    with open('error.txt', 'a') as error_file:
        error_file.write(str(err) + '\n')




Connection established.


In [14]:
import os
import logging

block = '5905903'  # Change this number if you want to load a different file
file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/{block}.json'

try:
    if os.path.exists(file_path):
        print(f"File '{file_path}' exists. Proceeding with further actions.")
        # Add your additional code here

        # Configure logging to write to both log.txt and error.txt
        logging.basicConfig(filename='log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.info(f"File '{file_path}' exists. Proceeding with further actions.")

    else:
        print(f"File '{file_path}' does not exist.")
        # Log the error message to error.txt
        with open('error.txt', 'a') as error_file:
            error_file.write(f"File '{file_path}' does not exist.\n")

except Exception as e:
    print(f"An error occurred: {e}")
    # Log the error message to error.txt
    with open('error.txt', 'a') as error_file:
        error_file.write(f"An error occurred: {e}\n")


File 'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/5905903.json' exists. Proceeding with further actions.


In [15]:
# This section of code uses json and pandas to clean json file and get it ready to sort it into a tabular format to prepare for postgres
import pandas as pd
import json
# Specify the file path

with open(file_path, 'r') as json_file:
    json_data = json.load(json_file)

# Slight modifications made to make data postgres friendly
def replace_empty_strings(item):
    return "N/A" if item == "null" else item

for field in ["sender", "receiver", "memo"]:
    json_data["tx"]["body"]["messages"][0][field] = replace_empty_strings(json_data["tx"]["body"]["messages"][0][field])
# This section of code calls the paths of all the attributes needs for the postgres table
df_messages = pd.json_normalize(json_data["tx"]["body"]["messages"])
df_auth_info = pd.json_normalize(json_data["tx"]["auth_info"]["signer_infos"])
df_fee_info = pd.json_normalize(json_data["tx"]["auth_info"]["fee"]["amount"])
df_sig_info = pd.DataFrame({"signatures": json_data["tx"]["signatures"]})
df_gas_info = pd.DataFrame({"gas_limit": [json_data["tx"]["auth_info"]["fee"]["gas_limit"]]})
df_payer_info = pd.DataFrame({"payer": [json_data["tx"]["auth_info"]["fee"]["payer"]]})
df_granter_info = pd.DataFrame({"granter": [json_data["tx"]["auth_info"]["fee"]["granter"]]})
df_tip_info = pd.DataFrame({"tip": [json_data["tx"]["auth_info"]["tip"]]})

# Combine dataframes
df_combined = pd.concat(
    [df_messages, df_auth_info, df_fee_info, df_sig_info, df_gas_info, df_payer_info, df_granter_info, df_tip_info],
    axis=1)

# Display the combined dataframe
print(df_combined)

                                       @type source_port source_channel  \
0  /ibc.applications.transfer.v1.MsgTransfer    transfer      channel-5   

                                           sender  \
0  migaloo17elxaps3vgzq4nrturr55c0z3rsn32nyzg8zg4   

                                      receiver    timeout_timestamp memo  \
0  osmo17elxaps3vgzq4nrturr55c0z3rsn32ny88agtf  1710188661779000000        

  token.denom token.amount timeout_height.revision_number  ...  \
0      uwhale   1852877000                              0  ...   

                  public_key.@type  \
0  /cosmos.crypto.secp256k1.PubKey   

                                 public_key.key mode_info.single.mode   denom  \
0  A7JU+NEqpAsGErxxsDWu1IDtVFew2RNY5aIqvJQFZ0Kt      SIGN_MODE_DIRECT  uwhale   

   amount                                         signatures gas_limit payer  \
0  150000  DthFHlVbz5af6ZFRmamVOjLlqh/0s3NjQYDrRJ8HPMcOmF...    150000         

  granter   tip  
0          None  

[1 rows x 22 colum

In [16]:
# Creates tabular format
column_names = list(df_combined.columns)
# Slight modifications that needed to be made to prepare for postgres
def clean_column_name(col): # Special symbols can not be inside of column names
    return col.replace("@", "").replace(" ", "_").replace('.', '')

column_names_cleaned = [clean_column_name(col) for col in column_names]

message = json_data["tx"]["body"]["messages"][0]
table_name = message["@type"].split(".")[-1][-10:]
table_name = table_name.lower()  # This has to be specified for check_table_query to work

In [17]:

# Create a cursor


# Check if the table already exists
check_table_query = f"""
    SELECT EXISTS (
        SELECT 1
        FROM pg_tables
        WHERE schemaname = 'public' AND tablename = '{table_name}'
    );
"""

cur.execute(check_table_query)
table_exists = cur.fetchone()[0]  # Fetch the result of the query

if table_exists:
    print(f"Table '{table_name}' already exists. Data will be appended to the existing table.")
else:
    print(f"Table '{table_name}' does not exist. Creating a new table...")

Table 'sgtransfer' already exists. Data will be appended to the existing table.


In [18]:

# %%

if not table_exists:
    # Create the table with dynamic column names (matching cleaned attributes)
    create_table_query = f"""
        CREATE TABLE {table_name} (
            id SERIAL PRIMARY KEY,
            {', '.join(f"{column_names_cleaned[i]} VARCHAR(255)" for i in range(len(column_names)))}, block INTEGER NOT NULL
        );
    """
    cur.execute(create_table_query)
    print(f"Table '{table_name}' created successfully.")
else:
    print(f"Table '{table_name}' already exists. Data will be appended to the existing table.")

# Insert data into the table
for _, row in df_combined.iterrows():
    # Assuming you have already defined the variable block_number
    # Modify the INSERT query to include the block number
    insert_query = f"""
        INSERT INTO {table_name} ({', '.join(column_names_cleaned)}, block)
        VALUES ({', '.join(f"'{row.iloc[i]}'" for i in range(len(column_names)))}, {block});
    """
cur.execute(insert_query)

# Commit changes
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

# Display the resulting DataFrame
print(df_combined)


Table 'sgtransfer' already exists. Data will be appended to the existing table.
                                       @type source_port source_channel  \
0  /ibc.applications.transfer.v1.MsgTransfer    transfer      channel-5   

                                           sender  \
0  migaloo17elxaps3vgzq4nrturr55c0z3rsn32nyzg8zg4   

                                      receiver    timeout_timestamp memo  \
0  osmo17elxaps3vgzq4nrturr55c0z3rsn32ny88agtf  1710188661779000000        

  token.denom token.amount timeout_height.revision_number  ...  \
0      uwhale   1852877000                              0  ...   

                  public_key.@type  \
0  /cosmos.crypto.secp256k1.PubKey   

                                 public_key.key mode_info.single.mode   denom  \
0  A7JU+NEqpAsGErxxsDWu1IDtVFew2RNY5aIqvJQFZ0Kt      SIGN_MODE_DIRECT  uwhale   

   amount                                         signatures gas_limit payer  \
0  150000  DthFHlVbz5af6ZFRmamVOjLlqh/0s3NjQYDrRJ8HPMcOm